In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout


In [2]:
data = pd.read_csv("data.csv")

In [3]:
pd.set_option('display.max_colwidth',300)

In [4]:
data.head()

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-41419190\r\n,Four ways Bob Corker skewered Donald Trump,"Image copyright Getty Images\nOn Sunday morning, Donald Trump went off on a Twitter tirade against a member of his own party.\nThis, in itself, isn't exactly huge news. It's far from the first time the president has turned his rhetorical cannons on his own ranks.\nThis time, however, his attacks...",1
1,https://www.reuters.com/article/us-filmfestival-london-lastflagflying/linklaters-war-veteran-comedy-speaks-to-modern-america-says-star-idUSKBN1CD0X2\r\n,"Linklater's war veteran comedy speaks to modern America, says star","LONDON (Reuters) - “Last Flag Flying”, a comedy-drama about Vietnam war veterans, will resonate with Trump’s America, despite, or perhaps because of, its period setting, actor Bryan Cranston said on Sunday after a screening at the London Film Festival.\nSet in the United States in December 2003 ...",1
2,https://www.nytimes.com/2017/10/09/us/politics/corkers-blast-at-trump-has-other-republicans-nodding-in-agreement.html?rref=collection%2Fsectioncollection%2Fpolitics\r\n,Trump’s Fight With Corker Jeopardizes His Legislative Agenda,"The feud broke into public view last week when Mr. Corker said that Mr. Trump’s advisers were guarding against “chaos.” The president retaliated on Sunday by saying the retiring senator “didn’t have the guts to run” for another term. Mr. Corker responded on Twitter an hour later, saying that the...",1
3,https://www.reuters.com/article/us-mexico-oil-cardenas/egypts-cheiron-wins-tie-up-with-pemex-for-mexican-onshore-oil-field-idUSKBN1C92EU\r\n,Egypt's Cheiron wins tie-up with Pemex for Mexican onshore oil field,"MEXICO CITY (Reuters) - Egypt’s Cheiron Holdings Limited won the rights to partner with Mexican national oil company Pemex on its onshore Cardenas-Mora project, the industry regulator said on Wednesday.\nThe tie-up marks only the second joint venture between the Pemex and a equity partner since ...",1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/snl-jason-aldean-las-vegas-tom-petty-orig.cnn/video/playlists/cant-miss/\n,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performing as the Las Vegas shooting began, did the ""Saturday Night Live"" cold open with a tribute to the victims and their families by singing Tom Petty's ""I Won't Back Down.""",1


In [5]:
data[data['Label']==1].head()

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-41419190\r\n,Four ways Bob Corker skewered Donald Trump,"Image copyright Getty Images\nOn Sunday morning, Donald Trump went off on a Twitter tirade against a member of his own party.\nThis, in itself, isn't exactly huge news. It's far from the first time the president has turned his rhetorical cannons on his own ranks.\nThis time, however, his attacks...",1
1,https://www.reuters.com/article/us-filmfestival-london-lastflagflying/linklaters-war-veteran-comedy-speaks-to-modern-america-says-star-idUSKBN1CD0X2\r\n,"Linklater's war veteran comedy speaks to modern America, says star","LONDON (Reuters) - “Last Flag Flying”, a comedy-drama about Vietnam war veterans, will resonate with Trump’s America, despite, or perhaps because of, its period setting, actor Bryan Cranston said on Sunday after a screening at the London Film Festival.\nSet in the United States in December 2003 ...",1
2,https://www.nytimes.com/2017/10/09/us/politics/corkers-blast-at-trump-has-other-republicans-nodding-in-agreement.html?rref=collection%2Fsectioncollection%2Fpolitics\r\n,Trump’s Fight With Corker Jeopardizes His Legislative Agenda,"The feud broke into public view last week when Mr. Corker said that Mr. Trump’s advisers were guarding against “chaos.” The president retaliated on Sunday by saying the retiring senator “didn’t have the guts to run” for another term. Mr. Corker responded on Twitter an hour later, saying that the...",1
3,https://www.reuters.com/article/us-mexico-oil-cardenas/egypts-cheiron-wins-tie-up-with-pemex-for-mexican-onshore-oil-field-idUSKBN1C92EU\r\n,Egypt's Cheiron wins tie-up with Pemex for Mexican onshore oil field,"MEXICO CITY (Reuters) - Egypt’s Cheiron Holdings Limited won the rights to partner with Mexican national oil company Pemex on its onshore Cardenas-Mora project, the industry regulator said on Wednesday.\nThe tie-up marks only the second joint venture between the Pemex and a equity partner since ...",1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/snl-jason-aldean-las-vegas-tom-petty-orig.cnn/video/playlists/cant-miss/\n,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performing as the Las Vegas shooting began, did the ""Saturday Night Live"" cold open with a tribute to the victims and their families by singing Tom Petty's ""I Won't Back Down.""",1


In [6]:
data[data['Label']==0].head()

,URLs,Headline,Body,Label
5,http://beforeitsnews.com/sports/2017/09/jetnation-fanduel-league-week-4-2701323.html\n\n,JetNation FanDuel League; Week 4,"JetNation FanDuel League; Week 4\n% of readers think this story is Fact. Add your two cents.\n(Before It's News)\nOur FanDuel league is back again this week. Here are the details:\n$900 in total prize money. $250 to the winner. $10 to enter.\nRemember this is a one week league, pick your lineup ...",0
10,https://www.activistpost.com/2017/09/u-s-president-donald-trump-quietly-signs-law-allow-warrant-less-searches-parts-va-dc-md.html\r\n,"U.S. President Donald Trump Quietly Signs Law To Allow Warrantless Searches In Parts Of VA, DC and MD","By Aaron Kesel\nAs former White House chief of staff and current Chicago Mayor Rahm Emanuel has said: “You never let a serious crisis go to waste.” While everyone was worried about the hurricanes, U.S. President Donald Trump signed away the Fourth Amendment buried in a new bill. He set a precede...",0
11,http://beforeitsnews.com/sports/2017/10/2017-fantasy-football-team-defense-rankings-week-5-2701900.html\n\n,2017 Fantasy Football Team Defense Rankings - Week 5,"2017 Fantasy Football Team Defense Rankings – Week 5\n% of readers think this story is Fact. Add your two cents.\n(Before It's News)\n.\nHere are the 2017 Lester’s Legends Week 5 team defense rankings.\n.\nBye Week: Atlanta, Denver, New Orleans, Washington\n.\n1. Minnesota Vikings\n2. Seattle Se...",0
12,http://beforeitsnews.com/sports/2017/09/just-shut-up-play-some-damn-baseball-2701059.html\r\n,Just Shut Up & Play Some Damn Baseball!!,"Just Shut Up & Play Some Damn Baseball!!\n(Before It's News)\nLook, plain and simple I am over this “Take a knee” bullshit in my opinion.\nKyle Terada-USA TODAY Sports\nI’m not writing to debate with anyone on this. Everyone has a right to their own opinion and to protest….I get it. But I also h...",0
14,http://beforeitsnews.com/sports/2017/10/107-chuck-axed-hbd-brickyard-adam-moonlight-ace-chuck-as-season-finales-2702154.html\r\n,"10/7: Chuck Axed; HBD Brickyard, Adam, Moonlight Ace & Chuck; AS; Season Finales...",A Potato Battery Can Light up a Room for Over a Month DIY (System That Can Be Used to Provide Rooms With LED-Powered Lighting for as Long as 40 days) +Video,0


In [7]:
data.nunique()

URLs        3352
Headline    2831
Body        2864
Label          2
dtype: int64

In [8]:
X = data.drop('Label',axis=1)
y = data['Label']

In [9]:
### Vocabulary size
voc_size=10000

#### **Onehot Representation**

In [10]:
messages=X.copy()
messages['Headline'][1]

"Linklater's war veteran comedy speaks to modern America, says star"

In [11]:
messages.reset_index(inplace=True)

In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
import nltk
import re
from nltk.corpus import stopwords

### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['Headline'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [14]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]

#### **Embedding Representation**

In [15]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 7217 6266 3360]
 [   0    0    0 ... 1636 1453 3647]
 [   0    0    0 ... 5237 7888 3962]
 ...
 [   0    0    0 ... 3360 6249 6429]
 [   0    0    0 ... 9003 1672 5808]
 [   0    0    0 ... 9025 9471 5244]]


In [16]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 8087, 4116, 8619, 7133, 7217, 6266, 3360], dtype=int32)

In [17]:
## Creating CNN model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Dropout(0.3))
model.add(Conv1D(filters=64, kernel_size=5, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            400000    
                                                                 
 dropout (Dropout)           (None, 20, 40)            0         
                                                                 
 conv1d (Conv1D)             (None, 20, 64)            12864     
                                                                 
 max_pooling1d (MaxPooling1  (None, 10, 64)            0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 10, 128)           41088     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 5, 128)            0         
 g1D)                                                   

In [18]:
len(embedded_docs),y.shape

(4328, (4328,))

In [19]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=42)

#### **Model Training**

In [21]:
### Finally Training
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=128)

Epoch 1/10
28/28 [==============================] - 3s 44ms/step - loss: 0.6849 - accuracy: 0.5384 - val_loss: 0.6661 - val_accuracy: 0.6316
Epoch 2/10
28/28 [==============================] - 1s 35ms/step - loss: 0.5763 - accuracy: 0.7080 - val_loss: 0.4339 - val_accuracy: 0.7956
Epoch 3/10
28/28 [==============================] - 1s 34ms/step - loss: 0.2592 - accuracy: 0.8879 - val_loss: 0.3522 - val_accuracy: 0.8568
Epoch 4/10
28/28 [==============================] - 1s 35ms/step - loss: 0.1126 - accuracy: 0.9573 - val_loss: 0.3572 - val_accuracy: 0.8776
Epoch 5/10
28/28 [==============================] - 1s 34ms/step - loss: 0.0650 - accuracy: 0.9775 - val_loss: 0.4288 - val_accuracy: 0.8730
Epoch 6/10
28/28 [==============================] - 1s 34ms/step - loss: 0.0425 - accuracy: 0.9853 - val_loss: 0.4489 - val_accuracy: 0.8776
Epoch 7/10
28/28 [==============================] - 1s 34ms/step - loss: 0.0187 - accuracy: 0.9951 - val_loss: 0.5180 - val_accuracy: 0.8741
Epoch 8/10
28

#### **Performance Metrics And Accuracy**

In [22]:
y_pred=model.predict(X_test)

28/28 [==============================] - 0s 3ms/step


In [23]:
# Convert y_pred to binary values
y_pred_binary = (y_pred > 0.5).astype(int)

In [24]:
from sklearn.metrics import confusion_matrix
# Calculate accuracy score
from sklearn.metrics import accuracy_score
print(f'accuracy_score: {accuracy_score(y_test, y_pred_binary)}')

accuracy_score: 0.8787528868360277
